### Finding influencers on social media
http://www.captaindatascience.com/top-fashion-influencers-instagram-twitter-an-algorithmic-approach-8

#### Basic steps
1. Finding top 10 influencers
2. access Twitter API
3. Find 20 most commonly followed by the top 10 influencers
4. repeat step 3 on the new 30, etc.


#### Initial setup

In [1]:
# Twitter API setup
import tweepy
import json
import csv
from tweepy import OAuthHandler

consumer_key = 'lun6TR6KpaISisFdGnQ5Eo8v5'
consumer_secret = 'hmwEtnfvTfI6CljEKKtIGjahG4NcFQvLBXhOnPyFHmAqNZ9fVV'
access_token = '3004335028-UKSgKFDbaBLNWTzXQFrBRDwVOKo0JR475KYY3LW'
access_secret = 'pA6MeW4NYsv3tL0MRvjI1oBqdUZc0os11gesdNVkeLpX2'
# consumer_key = "HKdRO2RsrerfwnBCYUjcUorpb"
# consumer_secret = "h3Roqkslf2wn3D6e8xBUt4kh4PBYG1WoUK4RAPgWZTUh24Qn6F"
# access_token = "451540968-QbTf67xnuYjlYBkf2MsKuH6SUh2OdJrD1n7EX7Gn"
# access_secret = "fhpAikh6eoByPoravJpn7BFMN8VrAUg10j7CgyaKBZw1R"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [2]:
# pandas setup
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [10]:
# open csv file
csvFile = open('initialtopinfluencers.csv', 'a')
# create csv writer
csvWriter = csv.writer(csvFile)


#### Getting the info from top 10 influencers

In [91]:
top_ten_influencers = ['chiaraferragni','aimeesong', 'wendynguyen', 'Kayture', 'SincerelyJules', 'rumineely', 
                       'garypeppergirl', 'GalMeetsGlam', 'BlairEadieBEE', 'bettyautier']

In [92]:
# get relevant user info and save to csv
def get_users_info(influencers):
    users_summary = []
    for influencer in influencers:
        user = api.get_user(screen_name = influencer)
        #save relevant info to csv
        user_summary = [user.id, user.screen_name, user.followers_count, user.friends_count]
        users_summary.append(user_summary)
        csvWriter.writerow(user_summary)
    return users_summary

In [30]:
# gets the summary info for top ten influencers
get_users_info(top_ten_influencers)

[[40090727, u'ChiaraFerragni', 273723, 153],
 [20298371, u'AIMEESONG', 62329, 359],
 [16252512, u'wendynguyen', 51012, 378],
 [157994424, u'Kayture', 36128, 493],
 [90963591, u'SincerelyJules', 56582, 436],
 [22830044, u'rumineely', 122310, 304],
 [112700733, u'garypeppergirl', 34180, 829],
 [282389552, u'GalMeetsGlam', 36106, 1142],
 [165572420, u'BlairEadieBEE', 52657, 313],
 [54859060, u'BettyAutier', 124774, 146]]

In [93]:
initial_influencers_df=pd.read_csv("initialtopinfluencers.csv", names=["id", "screen_name", 
                                                                       "follower_count", "friend_count"])
initial_influencers_df.head(10)

,id,screen_name,follower_count,friend_count
0,40090727,ChiaraFerragni,273723,153
1,20298371,AIMEESONG,62329,359
2,16252512,wendynguyen,51012,378
3,157994424,Kayture,36128,493
4,90963591,SincerelyJules,56582,436
5,22830044,rumineely,122310,304
6,112700733,garypeppergirl,34180,829
7,282389552,GalMeetsGlam,36106,1142
8,165572420,BlairEadieBEE,52657,313
9,54859060,BettyAutier,124774,146


In [126]:
sum(initial_influencers_df.friend_count)

4553

#### Get friends from top influencers

In [121]:
def get_friends_ids(user_id):
    ids = []
    page_count = 0
    print user_id
    for page in tweepy.Cursor(api.friends_ids, id=user_id).pages():
        page_count += 1
        print 'Getting page {} for friends ids'.format(page_count)
        ids.extend(page)
        time.sleep(60)
    return ids

In [122]:
all_friend_ids = []
for influencer_id in initial_influencers_df.id:
    print influencer_id
    
    # get all friends of the influencer
    ids = get_friends_ids(influencer_id)
    # add to our comprehensive list
    all_friend_ids = all_friend_ids + ids
    

40090727
40090727
Getting page 1 for friends ids
20298371
20298371
Getting page 1 for friends ids
16252512
16252512
Getting page 1 for friends ids
157994424
157994424
Getting page 1 for friends ids
90963591
90963591
Getting page 1 for friends ids
22830044
22830044
Getting page 1 for friends ids
112700733
112700733
Getting page 1 for friends ids
282389552
282389552
Getting page 1 for friends ids
165572420
165572420
Getting page 1 for friends ids
54859060
54859060
Getting page 1 for friends ids


In [123]:
len(all_friend_ids)

4553

In [4]:
all_friend_ids_saved = all_friend_ids

NameError: name 'all_friend_ids' is not defined

#### Sort friends by most common and get top ten

In [128]:
from collections import Counter
from itertools import groupby
# sort items given list test
sorted_list = [item for items, c in Counter(all_friend_ids).most_common() for item in [items] * c]
# top users
top_users = [key for key, group in groupby(sorted_list)]
# group by key values
new_array_to_display = [(key, len(list(group))) for key, group in groupby(sorted_list)]

In [129]:
new_array_to_display

[(19212009, 10),
 (228379737, 9),
 (64822927, 9),
 (17475920, 8),
 (21190774, 8),
 (17809182, 8),
 (90963591, 8),
 (24190981, 7),
 (73359920, 7),
 (15090453, 7),
 (16271952, 7),
 (437086963, 7),
 (109702885, 7),
 (16945578, 7),
 (35443315, 7),
 (20779908, 7),
 (22582248, 7),
 (136361303, 7),
 (18307364, 7),
 (20298371, 7),
 (54819492, 7),
 (21208444, 7),
 (250314584, 7),
 (19390810, 6),
 (33587619, 6),
 (38527593, 6),
 (19546942, 6),
 (19399719, 6),
 (15934926, 6),
 (25283065, 6),
 (204491797, 6),
 (67185748, 6),
 (44084633, 6),
 (157994424, 6),
 (25906700, 6),
 (31002374, 6),
 (21401466, 6),
 (183734502, 6),
 (20908109, 6),
 (24597041, 6),
 (32469566, 6),
 (34296742, 6),
 (28726228, 6),
 (73200694, 6),
 (7092102, 6),
 (250911289, 6),
 (44415931, 6),
 (22830044, 6),
 (24599923, 6),
 (92577188, 6),
 (153656769, 6),
 (141598384, 6),
 (24895193, 6),
 (46470446, 6),
 (21692557, 5),
 (51945792, 5),
 (15933910, 5),
 (20177423, 5),
 (153474021, 5),
 (129319986, 5),
 (21442110, 5),
 (34211097,

In [3]:
top_twenty_ids = top_users[0:20]
top_twenty_ids

NameError: name 'top_users' is not defined

#### Get user info

In [142]:
# open csv file
csvFileMore = open('moretopinfluencers.csv', 'a')
# create csv writer
csvWriterMore = csv.writer(csvFileMore)

In [143]:
# get relevant user info and save to csv
def get_users_info(influencers):
    users_summary = []
    for influencer in influencers:
        user = api.get_user(id = influencer)
        #save relevant info to csv
        user_summary = [user.id, user.screen_name, user.followers_count, user.friends_count]
        users_summary.append(user_summary)
        csvWriterMore.writerow(user_summary)
    return users_summary

In [144]:
# gets the summary info for top ten influencers
more_influencers = get_users_info(top_twenty_ids)

In [145]:
initial_influencers_df=pd.read_csv("moretopinfluencers.csv", names=["id", "screen_name", 
                                                                       "follower_count", "friend_count"])
initial_influencers_df.head(20)

,id,screen_name,follower_count,friend_count
0,19212009,WhoWhatWear,2101444,1594
1,228379737,thecoveteur,155119,2766
2,64822927,sea_of_shoes,101586,570
3,17475920,BagSnob,165187,888
4,21190774,tuulavintage,38864,415
5,17809182,MattiasSwenson,8124,119
6,90963591,SincerelyJules,56593,436
7,24190981,TeenVogue,2588637,2819
8,73359920,MaisonValentino,1656069,975
9,15090453,gerihirsch,16550,384


The only false positive I could find was Matthias Swenson

#### Cycle 2

In [3]:
more_influencers_df=pd.read_csv("moretopinfluencers.csv", names=["id", "screen_name", 
                                                                       "follower_count", "friend_count"])
more_influencers_df.head(20)

,id,screen_name,follower_count,friend_count
0,19212009,WhoWhatWear,2101444,1594
1,228379737,thecoveteur,155119,2766
2,64822927,sea_of_shoes,101586,570
3,17475920,BagSnob,165187,888
4,21190774,tuulavintage,38864,415
5,17809182,MattiasSwenson,8124,119
6,90963591,SincerelyJules,56593,436
7,24190981,TeenVogue,2588637,2819
8,73359920,MaisonValentino,1656069,975
9,15090453,gerihirsch,16550,384


##

In [17]:
# get relevant user info and save to csv
def get_users_bio(influencers):
    users_bio = []
    for influencer in influencers:
        user = api.get_user(id = influencer)
        #save relevant info to csv
        user_bio = user.description
        users_bio.append(user_bio)
    return users_bio
        
# gets the summary info for top ten influencers
more_bios = get_users_bio(more_influencers_df['id'])
more_bios

TweepError: [{u'message': u'Rate limit exceeded', u'code': 88}]

In [18]:
initial_influencers_df

,id,screen_name,follower_count,friend_count
0,19212009,WhoWhatWear,2101444,1594
1,228379737,thecoveteur,155119,2766
2,64822927,sea_of_shoes,101586,570
3,17475920,BagSnob,165187,888
4,21190774,tuulavintage,38864,415
5,17809182,MattiasSwenson,8124,119
6,90963591,SincerelyJules,56593,436
7,24190981,TeenVogue,2588637,2819
8,73359920,MaisonValentino,1656069,975
9,15090453,gerihirsch,16550,384


In [23]:
more_bios
key_words = ['fashion', 'style', 'bag', 'shoe']

[u'Fashion and style, decoded.',
 u'Behind-the-scenes & beyond. \u2728 Preorder The Coveteur: Private Spaces, Personal Style here: https://t.co/PGMeS0gV2n',
 u'Butter face\nhttp://t.co/IfrZLDbx5I',
 u'| Co-Founder Snob Essentials |BagChat: Vogue China/Vogue Mini | TV presenter | Wife, Mother, Bag Snob for life.',
 u'Travel & Fashion blogger. Not all those who wander are lost. Instagram @tuulavintage',
 u'Co-founder @bloglovin',
 u'creative type so believe the hype.',
 u'Future tastemakers start here. Snapchat: teenvogue',
 u'VALENTINO official account. Founded in 1960 by Valentino Garavani, since 2008 under the creative guide of Maria Grazia Chiuri and Pierpaolo Piccioli.',
 u'A curious girl from Los Angeles sharing videos, ideas and inspiration on https://t.co/XrbPNoyNLF & https://t.co/hbeVLwOqXZ. Hiiiii!',
 u'The Downtown Romantic.',
 u'The official Twitter page of 3.1 Phillip Lim',
 u'snapchat: marcjacobs',
 u'The official Twitter page of Garance Dor\xe9, photographer, illustrator a

In [22]:
initial_influencers_df['description'] = more_bios
initial_influencers_df.head(20)

,id,screen_name,follower_count,friend_count,description
0,19212009,WhoWhatWear,2101444,1594,"Fashion and style, decoded."
1,228379737,thecoveteur,155119,2766,Behind-the-scenes & beyond. ✨ Preorder The Cov...
2,64822927,sea_of_shoes,101586,570,Butter face\nhttp://t.co/IfrZLDbx5I
3,17475920,BagSnob,165187,888,| Co-Founder Snob Essentials |BagChat: Vogue C...
4,21190774,tuulavintage,38864,415,Travel & Fashion blogger. Not all those who wa...
5,17809182,MattiasSwenson,8124,119,Co-founder @bloglovin
6,90963591,SincerelyJules,56593,436,creative type so believe the hype.
7,24190981,TeenVogue,2588637,2819,Future tastemakers start here. Snapchat: teenv...
8,73359920,MaisonValentino,1656069,975,VALENTINO official account. Founded in 1960 by...
9,15090453,gerihirsch,16550,384,A curious girl from Los Angeles sharing videos...


In [ ]:
for tweet in tweepy.Cursor(api.search, q='#yoursearch',result_type='popular').items(5):
    print(tweet)

In [24]:
stuff = api.user_timeline(screen_name = 'danieltosh', count = 100, include_rts = True)

print stuff

[Status(contributors=None, truncated=False, text=u'if you want it raw, uncut and 30 minutes longer i will need your credit card number. https://t.co/KZOYojTrZE #peoplepleaser', is_quote_status=False, in_reply_to_status_id=None, id=721896178724548608, favorite_count=1300, _api=<tweepy.api.API object at 0x109a19ed0>, author=User(follow_request_sent=False, has_extended_profile=False, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': False, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 25521487, u'profile_background_image_url_https': u'https://pbs.twimg.com/profile_background_images/12054191/toshbck.jpg', u'verified': True, u'profile_text_color': u'666666', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/218283715/Daniel-Tosh---Shot_2-12976_normal.gif', u'profile_sidebar_fill_color': u'252429', u'entities': {u'url': {u'urls': [{u'url': u'http://t.co/qu6CkTVbHi', u'indices': [0, 22], u'expand

- continue iterating on this to get 90 influencers (and determine the false positive rate)
- we can do some text analysis on the descriptions or top tweets to further categorize people